Objectif :
Mettre en pratique les concepts de modèle encoder-decoder en construisant un modèle capable de générer un résumé automatique d'un texte.


1.	Téléchargez et préparez un jeu de données : Utilisez un jeu de données contenant des textes longs ainsi que leurs résumés. Par exemple, vous pouvez utiliser le jeu de données CNN/Daily Mail disponible sur Hugging Face Datasets. Préparez les données en suivant ces étapes :
•	Charger les textes et les résumés associés.
•	Effectuer un prétraitement (nettoyage du texte, suppression des caractères spéciaux, etc.).
•	Diviser les données en ensembles d’entraînement, de validation et de test.


Charger les textes et les résumés associés.

In [7]:
import certifi
import os

os.environ['CURL_CA_BUNDLE'] = certifi.where()


In [8]:
from datasets import load_dataset

dataset = load_dataset("abisee/cnn_dailymail", "3.0.0")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

NonMatchingSplitsSizesError: [{'expected': SplitInfo(name='train', num_bytes=1261703785, num_examples=287113, shard_lengths=None, dataset_name=None), 'recorded': SplitInfo(name='train', num_bytes=3785111355, num_examples=861339, shard_lengths=[115705, 115704, 115704, 115705, 121408, 115705, 115704, 45704], dataset_name='parquet')}, {'expected': SplitInfo(name='validation', num_bytes=57732412, num_examples=13368, shard_lengths=None, dataset_name=None), 'recorded': SplitInfo(name='validation', num_bytes=173197236, num_examples=40104, shard_lengths=None, dataset_name='parquet')}, {'expected': SplitInfo(name='test', num_bytes=49925732, num_examples=11490, shard_lengths=None, dataset_name=None), 'recorded': SplitInfo(name='test', num_bytes=149777196, num_examples=34470, shard_lengths=None, dataset_name='parquet')}]

In [10]:
print(dataset)

NameError: name 'dataset' is not defined

In [12]:
dataset = load_dataset("cnn_dailymail", "3.0.0", streaming=True)
for split in ['train', 'validation', 'test']:
    split_data = dataset[split]
    count = 0
    for _ in split_data:
        count += 1
    print(f"{split} split has {count} examples")


NotImplementedError: Loading a streaming dataset cached in a LocalFileSystem is not supported yet.

In [5]:
pip install --upgrade pyOpenSSL


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Program Files\PostgreSQL\16\ssl\certs\ca-bundle.crt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from datasets import load_dataset

# Charger le jeu de données CNN/Daily Mail
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Visualisation des données
print(dataset)


In [ ]:
pip install datasets

In [10]:
from datasets import load_dataset

# Charger un split spécifique pour voir si ça fonctionne
dataset = load_dataset("cnn_dailymail", "1.0.0", split='train[:1%]', cache_dir="F:/huggingface_datasets")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

NonMatchingSplitsSizesError: [{'expected': SplitInfo(name='train', num_bytes=1261703785, num_examples=287113, shard_lengths=None, dataset_name=None), 'recorded': SplitInfo(name='train', num_bytes=3785111355, num_examples=861339, shard_lengths=[115705, 115704, 115704, 115705, 121408, 115705, 115704, 45704], dataset_name='parquet')}, {'expected': SplitInfo(name='validation', num_bytes=57732412, num_examples=13368, shard_lengths=None, dataset_name=None), 'recorded': SplitInfo(name='validation', num_bytes=173197236, num_examples=40104, shard_lengths=None, dataset_name='parquet')}, {'expected': SplitInfo(name='test', num_bytes=49925732, num_examples=11490, shard_lengths=None, dataset_name=None), 'recorded': SplitInfo(name='test', num_bytes=149777196, num_examples=34470, shard_lengths=None, dataset_name='parquet')}]